In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

**Importing Data**

In [2]:
train = pd.read_excel('Data_Train.xlsx')
test = pd.read_excel('Data_Test.xlsx')
train.shape

(6237, 9)

In [3]:
test.shape

(1560, 8)

In [4]:
train.BookCategory.value_counts()

Action & Adventure                      818
Crime, Thriller & Mystery               723
Biographies, Diaries & True Accounts    596
Language, Linguistics & Writing         594
Comics & Mangas                         583
Romance                                 560
Humour                                  540
Arts, Film & Photography                517
Computing, Internet & Digital Media     510
Sports                                  471
Politics                                325
Name: BookCategory, dtype: int64

In [5]:
train.Author.sort_values()

2146       0, Butterfield, Ngondi, Kerr
3743    0, Jonathan Law, Richard Rennie
986                     0, Kerr, Wright
2075                     0, Rennie, Law
3296                          0, Speake
                     ...               
4724                Zygmunt Miloszewski
2850                              dodie
747                            r.h. Sin
5083                renu and neena kaul
5662                       sister Jesme
Name: Author, Length: 6237, dtype: object

In [6]:
train.Reviews[1].split()[0]

'3.9'

In [7]:
train.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


**Combining Dataset(Train + Test)** - _for cleaning and feature engineering_

In [8]:
train.Price.isnull().sum()

0

In [ ]:
#train.drop_duplicates(['Title','Author','Edition'],inplace=True)

In [ ]:
#train[train.duplicated(['Title','Author','Edition'],keep=False)].sort_values('Title')

In [10]:
train = train[train.Price<10000]

In [12]:
combined = pd.concat([train, test], sort=False)
combined.reset_index(drop=True, inplace=True)
combined.tail()
print(combined.shape)

(7794, 9)


In [13]:
combined.Author.value_counts().count()

4369

In [14]:
pd.DataFrame(combined.Author.value_counts()).sort_index()

,Author
"0, Butterfield, Ngondi, Kerr",1
"0, Jonathan Law",1
"0, Jonathan Law, Richard Rennie",1
"0, Kerr, Wright",1
"0, Rennie, Law",1
...,...
Zygmunt Miloszewski,1
dodie,1
r.h. Sin,2
renu and neena kaul,1


In [15]:
from collections import Counter
Counter(combined.Author).most_common()

[('Agatha Christie', 87),
 ('Ladybird', 63),
 ('DK', 61),
 ('Albert Uderzo', 48),
 ('Herge', 40),
 ('Nora Roberts', 40),
 ('James Patterson', 39),
 ('Bill Watterson', 36),
 ('John Grisham', 34),
 ('Sidney Sheldon', 33),
 ('P.G. Wodehouse', 33),
 ('Clive Cussler', 30),
 ('Sophie Kinsella', 27),
 ('David Baldacci', 26),
 ('Wilbur Smith', 26),
 ('Stephen King', 26),
 ('Danielle Steel', 25),
 ('Lee Child', 25),
 ('George R.R. Martin', 23),
 ('Jeffrey Archer', 22),
 ("Louis L'Amour", 21),
 ('Frederick Forsyth', 21),
 ('Oliver Bowden', 20),
 ('Dreamland Publications', 20),
 ('Michael Crichton', 19),
 ('Matthew Reilly', 19),
 ('Akira Toriyama', 19),
 ('Ruskin Bond', 18),
 ('Geronimo Stilton', 17),
 ('René Goscinny, Albert Uderzo', 17),
 ('Neil Gaiman', 17),
 ('Various', 16),
 ('Robert Ludlum', 16),
 ('Alistair MacLean', 15),
 ('Dan Brown', 15),
 ('Oxford Dictionaries', 15),
 ('James Rollins', 15),
 ('Ken Follett', 15),
 ('Daniel Silva', 14),
 ('Haruki Murakami', 14),
 ('Trinity College Lond',

In [16]:
combined.shape

(7794, 9)

# Feature Cleaning & Extraction

In [ ]:
combined['Title'] = combined['Title'].str.lower()


**Splitting Edition** - *to Edition Binding type and other feature*

In [17]:
combined.Edition.value_counts()

Paperback,– 5 Oct 2017               60
Paperback,– 2016                     58
Paperback,– 2017                     47
Paperback,– 2019                     37
Paperback,– 2013                     35
                                     ..
Paperback,– 19 Dec 2008               1
Paperback,– Student Edition, 2014     1
Paperback,– Illustrated, Jul 2014     1
Paperback,– 6 Jul 2015                1
Paperback,– 18 Apr 2017               1
Name: Edition, Length: 3879, dtype: int64

**Binning Edition Binding** - *combined edition binding ( with occurence < 9 --> "other" )*

In [18]:
Counter(combined.Edition).most_common()

[('Paperback,– 5 Oct 2017', 60),
 ('Paperback,– 2016', 58),
 ('Paperback,– 2017', 47),
 ('Paperback,– 2019', 37),
 ('Paperback,– 2013', 35),
 ('Paperback,– 1 Jan 2013', 30),
 ('Paperback,– 2012', 25),
 ('Paperback,– 2015', 24),
 ('Hardcover,– 2 Aug 2009', 23),
 ('Paperback,– 2014', 23),
 ('Paperback,– 2018', 22),
 ('Paperback,– 14 Oct 2000', 20),
 ('Paperback,– 2010', 20),
 ('Paperback,– 2011', 19),
 ('Paperback,– 1 Apr 2019', 18),
 ('Paperback,– 1 Sep 2011', 17),
 ('Paperback,– 5 Sep 2005', 16),
 ('Paperback,– 25 Apr 2019', 16),
 ('Paperback,– 30 Oct 2017', 14),
 ('Paperback,– 2 Aug 2012', 13),
 ('Paperback,– 4 Oct 2016', 12),
 ('Paperback,– 30 Jun 2015', 12),
 ('Paperback,– 23 Mar 2017', 12),
 ('Paperback,– 2008', 12),
 ('Paperback,– 1 Jan 2009', 11),
 ('Paperback,– Import, 14 Dec 2017', 11),
 ('Paperback,– 7 Apr 2011', 11),
 ('Paperback,– 7 Oct 2004', 11),
 ('Paperback,– 29 May 2018', 11),
 ('Paperback,– 6 Sep 2016', 10),
 ('Paperback,– 27 Aug 2013', 10),
 ('Paperback,– 20 Apr 2016'

In [19]:
for ed in combined['Edition']:
    if ed.find("Hardcover")!=-1: print(ed)
#train['Edition'][3].split(',')

Hardcover,– 10 Oct 2006
Hardcover,– Import, 1 Mar 2018
Hardcover,– 8 Mar 2018
Hardcover,– 24 Nov 2018
Hardcover,– Deckle Edge, 18 Oct 2011
Hardcover,– 15 Sep 2014
Hardcover,– 10 May 2016
Hardcover,– 28 Sep 2013
Hardcover,– 8 Jul 2015
Hardcover,– 2019
Hardcover,– 4 Oct 2016
Hardcover,– 25 Dec 2016
Hardcover,– 2 Mar 1999
Hardcover,– 1 Sep 2016
Hardcover,– 21 Jan 2019
Hardcover,– 20 Aug 2018
Hardcover,– Import, 5 Jul 2018
Hardcover,– Import, 25 Mar 2019
Hardcover,– 5 Nov 2015
Hardcover,– 19 May 2019
Hardcover,– 4 Sep 2014
Hardcover,– 1 Sep 2016
Hardcover,– 16 Jan 2016
Hardcover,– 25 Aug 2002
Hardcover,– 1 Apr 2009
Hardcover,– 17 Dec 2018
Hardcover,– 8 Jul 2016
Hardcover,– 7 Sep 2006
Hardcover,– Import, 16 Mar 2018
Hardcover,– 26 Apr 2011
Hardcover,– 22 Jul 2018
Hardcover,– 18 Aug 2009
Hardcover,– 2 Aug 2009
Hardcover,– 14 Jun 2018
Hardcover,– 2016
Hardcover,– 21 Feb 2019
Hardcover,– 26 Sep 2017
Hardcover,– Illustrated, 24 Sep 2014
Hardcover,– 24 Dec 2018
Hardcover,– 19 Nov 2013
Hardcover,

In [20]:
for ed in combined['Edition']:
    if ed.find("Audiobook")!=-1: print(ed)
#train['Edition'][3].split(',')

Paperback,– Audiobook, 28 Feb 2003
Paperback,– Abridged, Audiobook, Box set
Hardcover,– Abridged, Audiobook, Box set
Hardcover,– Audiobook, Unabridged
Paperback,– Abridged, Audiobook, Large Print
Paperback,– Abridged, Audiobook, Box set
Paperback,– Abridged, Audiobook, Box set
Paperback,– Illustrated, Large Print, Audiobook
Paperback,– Audiobook, 16 Oct 2014
Paperback,– Abridged, Audiobook, Box set
Paperback,– Audiobook, 10 Aug 2018
Paperback,– Audiobook, Illustrated, Large Print
Paperback,– Audiobook, Box set, Large Print
Paperback,– Audiobook, 16 Jun 2016
Paperback,– Abridged, Audiobook, Large Print
Hardcover,– Abridged, Audiobook, Box set
Paperback,– Audiobook, 16 Jun 2016
Paperback,– Abridged, Audiobook, Box set


In [28]:
combined.tail()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,EditionBinding,EditionType1
7789,100 Things Every Designer Needs to Know About ...,Susan Weinschenk,"Paperback,– 14 Apr 2011",5.0 out of 5 stars,4 customer reviews,We design to elicit responses from people. We ...,Design,"Computing, Internet & Digital Media",NaN,Paperback,14 Apr 2011
7790,"Modern Letter Writing Course: Personal, Busine...",ARUN SAGAR,"Paperback,– 8 May 2013",3.6 out of 5 stars,13 customer reviews,"A 30-day course to write simple, sharp and att...",Children's Reference (Books),"Biographies, Diaries & True Accounts",NaN,Paperback,8 May 2013
7791,The Kite Runner Graphic Novel,Khaled Hosseini,"Paperback,– 6 Sep 2011",4.0 out of 5 stars,5 customer reviews,The perennial bestseller-now available as a se...,Humour (Books),Humour,NaN,Paperback,6 Sep 2011
7792,Panzer Leader (Penguin World War II Collection),Heinz Guderian,"Paperback,– 22 Sep 2009",3.5 out of 5 stars,3 customer reviews,Heinz Guderian - master of the Blitzkrieg and ...,United States History,"Biographies, Diaries & True Accounts",NaN,Paperback,22 Sep 2009
7793,Complete Spanish Step-by-Step,Barbara Bregstein,"Paperback,– 16 Sep 2016",4.5 out of 5 stars,2 customer reviews,Learn Spanish with the most convenient and eff...,Dictionaries,"Language, Linguistics & Writing",NaN,Paperback,16 Sep 2016


In [29]:
combined[['EditionBinding','EditionType1']] = combined['Edition'].str.split(',– ',expand=True)
combined.tail(2)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,EditionBinding,EditionType1
7792,Panzer Leader (Penguin World War II Collection),Heinz Guderian,"Paperback,– 22 Sep 2009",3.5 out of 5 stars,3 customer reviews,Heinz Guderian - master of the Blitzkrieg and ...,United States History,"Biographies, Diaries & True Accounts",NaN,Paperback,22 Sep 2009
7793,Complete Spanish Step-by-Step,Barbara Bregstein,"Paperback,– 16 Sep 2016",4.5 out of 5 stars,2 customer reviews,Learn Spanish with the most convenient and eff...,Dictionaries,"Language, Linguistics & Writing",NaN,Paperback,16 Sep 2016


In [30]:
combined.EditionType1.value_counts()

5 Oct 2017                 60
2016                       60
2017                       50
2019                       40
2013                       37
                           ..
31 May 2013                 1
20 Feb 2014                 1
1 Aug 1989                  1
Unabridged, 18 Sep 2009     1
5 Apr 1995                  1
Name: EditionType1, Length: 3329, dtype: int64

In [31]:
combined['EditionBinding'].value_counts()

Paperback                6458
Hardcover                1054
Mass Market Paperback     200
Sheet music                32
Flexibound                 19
Cards                       9
Spiral-bound                6
Tankobon Softcover          3
Loose Leaf                  3
(German),Paperback          1
(Kannada),Paperback         1
Plastic Comb                1
(French),Paperback          1
Board book                  1
Leather Bound               1
(Chinese),Paperback         1
(Spanish),Paperback         1
Perfect Paperback           1
Library Binding             1
Name: EditionBinding, dtype: int64

In [38]:
edition_binding_dict = combined['EditionBinding'].value_counts().to_dict()

edition_binding_dict

{'Paperback': 6458,
 'Hardcover': 1054,
 'Mass Market Paperback': 200,
 'Sheet music': 32,
 'other': 31,
 'Flexibound': 19}

In [39]:
combined['EditionBinding'] = combined['EditionBinding'].apply(lambda x: (x if edition_binding_dict[x] > 9 else 'other'))


In [40]:
combined['EditionBinding'].value_counts()

Paperback                6458
Hardcover                1054
Mass Market Paperback     200
Sheet music                32
other                      31
Flexibound                 19
Name: EditionBinding, dtype: int64

**Splitting Edition remainder part** - *extracting edition date and edition type*

In [41]:
Counter(combined.EditionType1).most_common()

[('5 Oct 2017', 60),
 ('2016', 60),
 ('2017', 50),
 ('2019', 40),
 ('2013', 37),
 ('2015', 35),
 ('1 Jan 2013', 31),
 ('2012', 25),
 ('2014', 25),
 ('2011', 24),
 ('2 Aug 2009', 23),
 ('2010', 23),
 ('2018', 22),
 ('25 Apr 2019', 21),
 ('14 Oct 2000', 20),
 ('1 Sep 2011', 19),
 ('1 Apr 2019', 18),
 ('4 Oct 2016', 17),
 ('5 Sep 2005', 16),
 ('30 Oct 2017', 14),
 ('27 Aug 2013', 14),
 ('1 Jan 2009', 13),
 ('15 Sep 2015', 13),
 ('30 Jun 2015', 13),
 ('6 Jun 2017', 13),
 ('18 Oct 2016', 13),
 ('2008', 13),
 ('2 Aug 2012', 13),
 ('1 Jul 2017', 13),
 ('1 Sep 2016', 12),
 ('6 Sep 2016', 12),
 ('25 Oct 2016', 12),
 ('23 Mar 2017', 12),
 ('7 Apr 2011', 12),
 ('1 Mar 2016', 11),
 ('10 Jul 2018', 11),
 ('1 Sep 2015', 11),
 ('3 Nov 2015', 11),
 ('Import, 14 Dec 2017', 11),
 ('3 Nov 2016', 11),
 ('7 Oct 2004', 11),
 ('2009', 11),
 ('29 May 2018', 11),
 ('26 Mar 2013', 10),
 ('28 Oct 2014', 10),
 ('14 Jun 2018', 10),
 ('10 Oct 2017', 10),
 ('13 Oct 2015', 10),
 ('20 Apr 2016', 10),
 ('28 Jun 2018', 

In [ ]:
def split_edition_1(x):
    j_arr = []
    date = ''

    for j in x.split(', '):
        if not any(k.isnumeric() for k in j):
            j_arr.append(j.strip())
        else:
            date = j

    if ''.join(j_arr) != '':
        ed = ', '.join(j_arr)
    else:
        ed = 'other'

    if ed != 'Import' and ed != 'Illustrated' and ed \
        != 'Special Edition' and ed != 'Unabridged' and ed \
        != 'Student Edition' and ed != 'Box set' and ed \
        != 'International Edition' and ed != 'Abridged':
        ed_ret = 'other'
    else:
        ed_ret = ed

    return (ed_ret, date)

In [ ]:
combined['EditionType'],combined['EditionDate'] = \
    zip(*combined['EditionType1'].apply(split_edition_1))

In [ ]:
combined.head(2)

**Splitting Edition date** - *extracting Month & Year*

In [ ]:
def split_edition_date(x):
    (mon, year) = ('', '')
    if len(x.split()) == 1:
        year = int(x)
    elif len(x.split()) == 2:
        mon = x.split()[0]
        year = int(x.split()[1])
    elif len(x.split()) == 3:
        mon = x.split()[1]
        year = int(x.split()[2])
    return (mon, year)

In [ ]:
combined['EditionMon'], combined['EditionYear'] = \
    zip(*combined['EditionDate'].apply(split_edition_date))

In [ ]:
combined.head(2)

**Binning Month** - *combining quaterly*

In [ ]:
def bin_edition_mon(x):
    x = x.lower()
    if x == 'jan' or x == 'feb' or x == 'mar':
        return 'first'
    elif x == 'apr' or x == 'may' or x == 'jun':
        return 'second'
    elif x == 'jul' or x == 'aug' or x == 'sep':
        return 'third'
    elif x == '':
        return ''
    else:
        return 'fourth'

*making columns to mark null values*

In [ ]:
combined['EditionMon'] = combined['EditionMon'].apply(bin_edition_mon)
combined['Mon_null'] = combined['EditionMon'].apply(lambda x: \
        ('not_null' if x != '' else 'null'))
combined['Year_null'] = combined['EditionYear'].apply(lambda x: \
        ('not_null' if x != '' else 'null'))

In [ ]:
combined.head(2)

**Imputing Month and Year** - *by most common values*

In [ ]:
combined[combined.EditionMon=='']

In [ ]:
combined['EditionMon'].replace('', combined['EditionMon'].mode()[0],
                               inplace=True)
combined['EditionYear'].replace('', combined['EditionYear'].mode()[0],
                                inplace=True)

In [ ]:
combined[combined.EditionMon=='']

In [ ]:
combined.head()

**Extracting Reviews & Ratings** - *converting to numerical data*

In [42]:
Counter(combined.Reviews).most_common()

[('5.0 out of 5 stars', 1751),
 ('4.0 out of 5 stars', 711),
 ('4.5 out of 5 stars', 626),
 ('4.6 out of 5 stars', 491),
 ('4.4 out of 5 stars', 488),
 ('4.3 out of 5 stars', 450),
 ('4.7 out of 5 stars', 412),
 ('4.2 out of 5 stars', 406),
 ('4.1 out of 5 stars', 381),
 ('3.9 out of 5 stars', 300),
 ('4.8 out of 5 stars', 272),
 ('3.8 out of 5 stars', 239),
 ('3.7 out of 5 stars', 203),
 ('3.0 out of 5 stars', 176),
 ('3.6 out of 5 stars', 145),
 ('3.5 out of 5 stars', 143),
 ('4.9 out of 5 stars', 98),
 ('3.4 out of 5 stars', 92),
 ('3.3 out of 5 stars', 74),
 ('1.0 out of 5 stars', 65),
 ('3.1 out of 5 stars', 57),
 ('3.2 out of 5 stars', 50),
 ('2.0 out of 5 stars', 44),
 ('2.9 out of 5 stars', 36),
 ('2.5 out of 5 stars', 23),
 ('2.7 out of 5 stars', 18),
 ('2.8 out of 5 stars', 12),
 ('2.3 out of 5 stars', 7),
 ('1.5 out of 5 stars', 6),
 ('2.6 out of 5 stars', 5),
 ('2.4 out of 5 stars', 5),
 ('2.2 out of 5 stars', 3),
 ('1.4 out of 5 stars', 2),
 ('1.7 out of 5 stars', 1),
 ('1

In [43]:
combined['Reviews'] = combined['Reviews'].apply(lambda x: float(x.split()[0]))

In [44]:
Counter(combined.Ratings).most_common()

[('1 customer review', 1326),
 ('2 customer reviews', 886),
 ('3 customer reviews', 613),
 ('4 customer reviews', 444),
 ('5 customer reviews', 379),
 ('6 customer reviews', 296),
 ('7 customer reviews', 231),
 ('8 customer reviews', 218),
 ('9 customer reviews', 185),
 ('10 customer reviews', 177),
 ('11 customer reviews', 153),
 ('12 customer reviews', 138),
 ('13 customer reviews', 123),
 ('15 customer reviews', 123),
 ('16 customer reviews', 119),
 ('14 customer reviews', 101),
 ('20 customer reviews', 81),
 ('17 customer reviews', 78),
 ('19 customer reviews', 71),
 ('22 customer reviews', 66),
 ('18 customer reviews', 65),
 ('30 customer reviews', 62),
 ('29 customer reviews', 56),
 ('21 customer reviews', 55),
 ('26 customer reviews', 53),
 ('27 customer reviews', 49),
 ('24 customer reviews', 47),
 ('23 customer reviews', 45),
 ('32 customer reviews', 45),
 ('33 customer reviews', 45),
 ('37 customer reviews', 41),
 ('25 customer reviews', 37),
 ('35 customer reviews', 36),
 ('

In [ ]:
combined['Ratings'] = combined['Ratings'].apply(lambda x: int(''.join(x.split()[0].split(','))))

In [45]:
combined.head(2)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,EditionBinding,EditionType1
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,Paperback,10 Mar 2016
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,Paperback,7 Nov 2012


# Feature Engineering 
*Engineering new features*

**Ratings and Reviews Ratio**

In [46]:
combined['RatingPerReview'] = round(combined['Ratings']/combined['Reviews'], 2)

TypeError: unsupported operand type(s) for /: 'str' and 'float'

In [ ]:
combined.Ratings.describe()

In [ ]:
%matplotlib inline
combined.Ratings.plot(kind='box');

In [ ]:
combined[combined.Ratings>2500]

In [ ]:
combined.head(2)

**Impact of Book Age on Reviews**

In [ ]:
combined['Review_Year_Impact'] = combined['Reviews'] * combined['EditionYear'].apply(lambda x: 2019 - x)

**Author Name Cleaning**

In [ ]:
combined.Author.head(100)

In [ ]:
author_replacements = {' & ':', ',"0":"other","2":"other",'A. P. J. Abdul Kalam':'A.P.J. Abdul Kalam','APJ Abdul Kalam':'A.P.J. Abdul Kalam','Agrawal P. K.': 'Agrawal P.K','Ajay K Pandey': 'Ajay K. Pandey','Aravinda Anantharaman': 'Aravinda Anatharaman','Arthur Conan Doyle': 'Sir Arthur Conan Doyle','B A Paris': 'B. A. Paris','E L James': 'E. L. James','E.L. James':'E. L. James','Eliyahu M Goldratt': 'Eliyahu M. Goldratt','Ernest Hemingway': 'Ernest Hemmingway','Frank Miler': 'Frank Miller','Fyodor Dostoevsky': 'Fyodor Dostoyevsky','George R R Martin': 'George R. R. Martin','George R.R. Martin':'George R. R. Martin','H. G. Wells': 'H.G. Wells','Johann Wolfgang Von Goethe': 'Johann Wolfgang von Goethe','John Le Carré': 'John le Carré','Judith McNaught': 'Judith Mcnaught','Keith Giffen': 'Kieth Giffen','Ken Hultgen': 'Ken Hultgren','Kentaro Miura': 'Kenturo Miura','Kohei Horikoshi': 'Kouhei Horikoshi','M.K Gandhi': 'M.K. Gandhi','Matthew K Manning': 'Matthew Manning','Michael Crichton': 'Micheal Crichton','N.K Aggarwala': 'N.K. Aggarwala','Oxford University Press (India)': 'Oxford University Press India','P D James': 'P. D. James','Paramahansa Yogananda': 'Paramhansa Yogananda','R K Laxman': 'R. K. Laxman','R.K. Laxman': 'R. K. Laxman','R. M. Lala': 'R.M. Lala','Raina Telgemaeier': 'Raina Telgemeier','Rajaraman': 'Rajaraman V','Rajiv M. Vijayakar': 'Rajiv Vijayakar','Ramachandra Guha': 'Ramchandra Guha','Rene Goscinny': 'René Goscinny','Richard P Feynman': 'Richard P. Feynman','S Giridhar': 'S. Giridhar','S Hussain Zaidi': 'S. Hussain Zaidi','S. A. Chakraborty': 'S. Chakraborty','Santosh Kumar K': 'Santosh Kumar K.',"S.C. Gupta" : "S. C. Gupta",'Shiv Prasad Koirala': 'Shivprasad Koirala','Shivaprasad Koirala': 'Shivprasad Koirala','Simone De Beauvoir': 'Simone de Beauvoir','Sir Arthur Conan Doyle': 'Arthur Conan Doyle',"Terry O' Brien": "Terry O'Brien",'Thich Nhat Hahn': 'Thich Nhat Hanh','Trinity College Lond': 'Trinity College London',"Trinity College London Press" : "Trinity College London",'Ursula K. Le Guin': 'Ursula Le Guin','Willard A Palmer': 'Willard A. Palmer','Willard Palmer': 'Willard A. Palmer','William Strunk Jr': 'William Strunk Jr.','Yashavant Kanetakr': 'Yashavant Kanetkar','Yashavant P. Kanetkar': 'Yashavant Kanetkar','Yashwant Kanetkar': 'Yashavant Kanetkar','et al': 'et al.',' et al': 'et al.','Peter Clutterbuck': ' Peter Clutterbuck','Scholastic': 'Scholastic ','Ullekh N. P.': 'Ullekh N.P.','Shalini Jain': 'Dr. Shalini Jain','Kevin Mitnick': 'Kevin D. Mitnick'}
combined['Author'] = combined['Author'].replace(author_replacements,regex=True)

**No. of Authors of a book**

In [47]:
combined['Authors_count'] = combined['Author'].apply(lambda x: \
        len(x.split(',')))

**Average Author reviews**

In [ ]:
author_avg_review_dict = round(combined[combined.Authors_count== 1]
                               .groupby('Author',sort=False)['Reviews']
                               .mean(), 2).to_dict()
author_avg_review_dict

In [ ]:
def check_author(x):
    reviews = []
    for name in x.split(', '):
        try:
            reviews.append(author_avg_review_dict[name])
        except:
            pass
    if len(reviews) != 0:
        return sum(reviews) / len(reviews)
    else:
        return ''

In [ ]:
combined['AuthorAvgReview'] = combined['Author'].apply(check_author)
combined['AuthorAvgReview'] = combined[['Reviews', 'AuthorAvgReview']]\
        .apply(lambda x: (x[0] if x[1] == '' else x[1]), axis=1)

**No. of Books from an Author**

In [ ]:
combined['Count_Author_Title'] = combined['Author'].map(combined.groupby('Author',sort=False)['Title'].apply(lambda x: len(x.unique())).to_dict())

**No. of occurences of a Title**
<br>
**Average:** 
- Book - Author Count
- Title - reviews

In [ ]:
combined['MEAN_Title_Authors_count'] = round(combined
                                            .groupby('Title',sort=False)['Authors_count']
                                            .transform('mean'), 2)

combined['MEAN_Ttle_Reviews'] = round(combined
                                      .groupby('Title',sort=False)['Reviews']
                                      .transform('mean'), 2)

combined['Title_count'] = combined.groupby('Title',sort=False)['Title']\
                                  .transform('count')

**Various Categories of a book**

In [ ]:
title_cat_dict = combined[combined.Authors_count == 1]\
                 .groupby('Title',sort=False)['BookCategory']\
                 .apply(lambda x: ', '.join(x)).to_dict()
combined['TitleCategories'] = combined['Title'].map(title_cat_dict)
combined['TitleCategories'] = combined[['BookCategory','TitleCategories']]\
                              .apply(lambda x: (x[0] if pd.isna(x[1]) else x[1]),axis=1)

**Various Genres of a book**

In [ ]:
title_genre_dict = combined[combined.Authors_count == 1]\
                   .groupby('Title',sort=False)['Genre']\
                   .apply(lambda x: ', '.join(x)).to_dict()
combined['TitleGenres'] = combined['Title'].map(title_genre_dict)
combined['TitleGenres'] = combined[['Genre', 'TitleGenres']]\
                          .apply(lambda x: (x[0] if pd.isna(x[1]) else x[1]), axis=1)

**Various Category books written by an author**

In [ ]:
author_cat_dict = combined[combined.Authors_count==1]\
                 .groupby('Author',sort=False)['BookCategory']\
                 .apply(lambda x: ', '.join(x)).to_dict()
combined['AuthorCategories'] = combined['Author'].map(author_cat_dict)
combined['AuthorCategories'] = combined[['BookCategory','AuthorCategories']]\
                               .apply(lambda x: x[0] if pd.isna(x[1]) else x[1],axis=1)

**Various Genre books written by an author**

In [ ]:
author_genre_dict = combined[combined.Authors_count==1]\
                    .groupby('Author',sort=False)['Genre']\
                    .apply(lambda x: ', '.join(x)).to_dict()
combined['AuthorGenres'] = combined['Author'].map(author_genre_dict)
combined['AuthorGenres'] = combined[['Genre','AuthorGenres']]\
                           .apply(lambda x: x[0] if pd.isna(x[1]) else x[1],axis=1)

In [ ]:
combined['TitleGenres'] = combined['TitleGenres'].str.replace(' & ',', ')
combined['AuthorGenres'] = combined['AuthorGenres'].str.replace(' & ',', ')
combined['Genre'] = combined['Genre'].str.replace(' & ',', ')

**Binning Edition Year ** - *by distribution over years*

In [ ]:
combined['EditionYearBin'] = pd.qcut(combined['EditionYear'],5,labels=False)

In [ ]:
combined.head()

## Dummy & Count Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
combined[['BookCategory','EditionBinding','EditionMon','EditionType','EditionYearBin',\
          'Mon_null','Year_null',]] = combined[['BookCategory','EditionBinding','EditionMon',\
    'EditionType','EditionYearBin','Mon_null','Year_null',]].apply(enc.fit_transform)

In [ ]:
combined.head(2)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

tc_vectorizer = CountVectorizer(lowercase=True, tokenizer=lambda x: \
                                 x.split(', '))
title_categories_vector = tc_vectorizer.fit_transform(combined['TitleCategories']).toarray()
data_title_categories = pd.DataFrame(data=title_categories_vector,
                      columns=tc_vectorizer.get_feature_names())

In [ ]:
ac_vectorizer = CountVectorizer(lowercase=True, 
                                 tokenizer=lambda x: x.split(', '))
author_categories_vector = ac_vectorizer.fit_transform(combined['AuthorCategories']).toarray()
data_author_categories = pd.DataFrame(data=author_categories_vector,
                      columns=ac_vectorizer.get_feature_names())

In [ ]:
tg_vectorizer = CountVectorizer(max_features=10, lowercase=True,
                                tokenizer=lambda x: x.split(', '))
title_genres_vector = tg_vectorizer.fit_transform(combined['TitleGenres']).toarray()
data_title_genres = pd.DataFrame(data=title_genres_vector,
                     columns=tg_vectorizer.get_feature_names())

In [ ]:
ag_vectorizer = CountVectorizer(max_features=10, lowercase=True,
                                    tokenizer=lambda x: x.split(', '))
author_genres_vector = ag_vectorizer.fit_transform(combined['AuthorGenres']).toarray()
data_author_genres = pd.DataFrame(data=author_genres_vector,
                         columns=ag_vectorizer.get_feature_names())

In [ ]:
title_vectorizer = CountVectorizer(max_features=10, lowercase=True)
title_vector = title_vectorizer.fit_transform(combined['Title']).toarray()
data_title = pd.DataFrame(data=title_vector,
                        columns=title_vectorizer.get_feature_names())

In [ ]:
vectorizer_author = CountVectorizer(max_features=10, lowercase=True,
                                    tokenizer=lambda x: x.split(', '))
vector_author = vectorizer_author.fit_transform(combined['Author']).toarray()
data_author = pd.DataFrame(data=vector_author,
                         columns=vectorizer_author.get_feature_names())

In [ ]:
vectorizer_genre = CountVectorizer(max_features=10,
                                   lowercase=True, tokenizer=lambda x: x.split(', '))
vector_genre = vectorizer_genre.fit_transform(combined['Genre']).toarray()
data_genre = pd.DataFrame(data=vector_genre,
                        columns=vectorizer_genre.get_feature_names())

In [ ]:
vectorizer_synopsis = CountVectorizer(max_features=10,
                                      stop_words='english', 
                                      strip_accents='ascii', 
                                      lowercase=True)
vector_synopsis = vectorizer_synopsis.fit_transform(combined['Synopsis']).toarray()
data_synopsis = pd.DataFrame(data=vector_synopsis,
                           columns=vectorizer_synopsis.get_feature_names())

In [ ]:
combined.drop(columns=[
    'Title',
    'Author',
    'Genre',
    'Synopsis',
    'Edition',
    'EditionDate',
    'EditionType1',
    'AuthorCategories',
    'AuthorGenres',
    'TitleGenres',
    'TitleCategories'
    ], inplace=True)

In [ ]:
print('No. of Features:',combined.shape[1])

**Feature correlations**

In [ ]:
data = pd.concat([
    combined,# dummy encoded features
    data_author, # author count encoded
    data_genre, # genre count encoded
    data_title, # title count encoded
    data_synopsis, # synopsis count encoded
   data_author_genres, # author_genres count encoded
   data_title_genres, # title_genres count encoded
    data_author_categories, # author_categories count encoded
    data_title_categories, # title_categories count encoded
    ], axis=1)
data.reset_index(drop=True, inplace=True)

In [ ]:
#feature correlations
corr = data.corr()
corr[corr.Price>0.01].Price.sort_values()

In [ ]:
data.shape  #  features count

In [ ]:
data.head()

In [ ]:
print(data.info());

# Train - Test Split

In [ ]:
train = data[data['Price'].notna()]
test = data[data['Price'].isna()]
test.drop(['Price'], axis=1, inplace=True)

In [ ]:
X = train.loc[:, train.columns != 'Price'].values
X = X.astype(float)

# Dependent Variable

y = np.log1p(train['Price'].values)
y = y.astype(float)

# Test - (Independent Variables)

test = test.loc[:].values
test = test.astype(float)

# Model Training

**Importing libraries**

In [ ]:
import xgboost as xgb
#import lightgbm as lgb
from sklearn.metrics import make_scorer
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, AdaBoostRegressor

**RMLSE scoring func**

In [ ]:
def score(y_true, y_pred):
    y_pred = np.exp(y_pred) - 1
    for i in range(len(y_pred)):
        if y_pred[i] < 0:
            y_pred[i] = 0
    y_true = np.exp(y_true) - 1
    error = np.square(np.log10(y_pred + 1) - np.log10(y_true
                      + 1)).mean() ** 0.5
    score = 1 - error
    return score

#### RandomForestRegressor

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
params = { 'random_state':range(0,4),
           'n_estimators':np.arange(10,100,10),
           'max_depth':np.arange(5,30,5),
           'max_features':['sqrt']
         }
rf = RandomizedSearchCV(RandomForestRegressor(),params,scoring=make_scorer(score,greater_is_better=True))
rf.fit(X,y)
rf.best_score_
#rf = RandomForestRegressor(random_state=0,n_estimators=150,max_features='sqrt')
#cvs = cross_val_score(rf, X, y, cv=5,verbose=2,n_jobs=-1,
#                      scoring=make_scorer(score,greater_is_better=True))
#print("Average Score:",np.mean(cvs))

In [ ]:
rf.best_params_

# SVR

In [ ]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf',gamma=0.003,C=1)

cvs = cross_val_score(svr, X, y, cv=5,verbose=2,n_jobs=-1,
                      scoring=make_scorer(score,greater_is_better=True))
print("Average Score:",np.mean(cvs))
# Not working with SVR

In [ ]:
rf1 = RandomForestRegressor(random_state=3,
 n_estimators=80,
 max_features='sqrt',       
 max_depth=30)

cvs = cross_val_score(rf1, X, y, cv=5,verbose=2,n_jobs=-1,
                      scoring=make_scorer(score,greater_is_better=True))
print("Average Score:",np.mean(cvs))

In [ ]:
rf1.fit(X,y)
pd.DataFrame(rf1.feature_importances_).sort_values(0,ascending=False)

#### LGBMRegressor

In [ ]:
lgbm = lgb.LGBMRegressor()

params = { 'random_state':range(0,4),
           'n_estimators':np.arange(10,100,10),
           'max_depth':np.arange(5,30,5),
           'max_features':['sqrt']
         }
lgbm = RandomizedSearchCV(lgbm,params,scoring=make_scorer(score,greater_is_better=True))
lgbm.fit(X,y)
print(lgbm.best_params_)
print(lgbm.best_score_)

#cvs = cross_val_score(lgbm, X, y, cv=5,verbose=2,n_jobs=-1,
#                      scoring=make_scorer(score,greater_is_better=True))
#print("Average Score:",np.mean(cvs))

#### XGBRegressor

In [ ]:
xgb = XGBRegressor( )

params = { 'random_state':range(0,4),
           'n_estimators':np.arange(10,100,10),
           'max_depth':np.arange(5,30,5),
           'max_features':['sqrt'],
          'colsample_bytree' : [0.6], 
           'objective':['reg:squarederror'],
          'learning_rate' : [0.1,0.2,0.05], 
          'max_depth' : [5,10,15,30], 
          'alpha' : [10]
         }
xgb = RandomizedSearchCV(xgb,params,scoring=make_scorer(score,greater_is_better=True))
xgb.fit(X,y)
xgb.best_score_

#### VotingRegressor

In [ ]:
vr = VotingRegressor([('rf', rf), ('xgb', xgb)])

cvs = cross_val_score(vr, X, y, cv=5,verbose=50,n_jobs=-1,
                        scoring=make_scorer(score,greater_is_better=True))
print("Average Score:",np.mean(cvs))

# Exporting Predictions

In [ ]:
vr.fit(X, y)
Y_pred2 = vr.predict(test)
Y_pred2 = np.exp(Y_pred2)-1

for i in range(len(Y_pred2)):
       if Y_pred2[i] < 0:
            Y_pred2[i] = 0

pd.DataFrame(Y_pred2, columns = ['Price']).to_excel("predictions.xlsx", index=False)